# Academy Tutorial

Academy is an agentic middleware platform designed to build, deploy, and operate autonomous agents at scientific scale. It provides the middleware layer between AI agents and real-world computing environments, enabling distributed execution, persistent state, and secure communication across institutions and resources. 

This tutorial provides a step by step introduction of Academy, from building a simple "Hello World" agent to deploying multi-agent systems on remote computing resources and integrating hosted LLMs. 

More information about Academy is available on [academyagents.org](https://academy-agents.org/)

#### Setup
Install Tutorial Package

The tutorial requires python>=3.12. We recommend completing the tutorial inside a virtual environment.
````
git clone git@github.com:academy-agents/academy-tutorial.git
cd academy-tutorial
python -m venv venv
. ./venv/bin/activate
pip install .
````

This will set up a local environment to complete the tutorial.

### 1. Hello World

We start with the simplest Academy program: defining an agent and interacting with it. A HelloWorld agent is created with a single action, `hello`, which returns a string. 

The program creates a local Academy manager, launches the agent locally, obtains a handle to it, and asynchronously invokes the action to produce and print the response. 

Academy follows an asynchronous execution model. When an agent is launched, a handle is returned. This handle acts as a reference to the running agent and provides the interface through which we communicate with it. Using the handle, we can asynchronously invoke actions exposed by the agent. Here, we call the `hello` action and use `await` to pause execution until the action completes and returns its result.

In [ ]:
from academy.agent import Agent
from academy.agent import action
from academy.manager import Manager
from academy.exchange import LocalExchangeFactory
from concurrent.futures import ThreadPoolExecutor

class HelloWorld(Agent):
   
    @action
    async def hello(self, value) -> str:
        return f"Hello {value}"


async with await Manager.from_exchange_factory(
        factory=LocalExchangeFactory(),
        executors=ThreadPoolExecutor(),
    ) as manager:
        agent_handle = await manager.launch(HelloWorld)

        print(await agent_handle.hello("World"))

### 2. Stateful agents: A Counter

We now create a stateful agent. This agent, a counter, stores a incrementable count and provides actions to increment and retrieve that count. 

#### Counter Agent

The Counter agent implements two basic actions:  increment a counter and retrieve the current value. When the agent is instantiated, its startup lifecycle hook initializes the internal state by resetting the count to 0.

In [ ]:
from academy.agent import Agent
from academy.agent import action

class Counter(Agent):
    count: int

    async def agent_on_startup(self) -> None:
        self.count = 0

    @action
    async def increment(self, value: int = 1) -> None:
        self.count += value

    @action
    async def get_count(self) -> int:
        return self.count

#### Counter Program

Now we use Academy to create the agent and interact with it. In this example, we use Academy’s ThreadPoolExecutor to launch the agent locally and a LocalExchange to enable communication between the user program and the agent.

Here, we call the `increment` action to update the counter and the `get_count` action to retrieve the agent’s current state. We see that the state is maintained between action invocations.

In [ ]:
from academy.manager import Manager
from academy.exchange import LocalExchangeFactory
from concurrent.futures import ThreadPoolExecutor

async with await Manager.from_exchange_factory(
        factory=LocalExchangeFactory(),
        executors=ThreadPoolExecutor(),
    ) as manager:
        # Launch a Counter agent locally
        agent_handle = await manager.launch(Counter)

        # Interact with the agent by invoking the get_count action
        count = await agent_handle.get_count()
        print("Count %s" % count)

        # Invoke the increment action
        await agent_handle.increment()

        # Invoke the get_count action
        count = await agent_handle.get_count()
        print("Count %s" % count)

###  3. Autonomous Agents: A Counting Counter

Academy allows agents to express autonomous action using the `@loop` decorator. This can be used to monitor and respond to changing state or environment.

We update the counter from above by adding an increment method to a loop that increments the value of count every one second. 

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from academy.agent import Agent
from academy.manager import Manager
from academy.agent import action
from academy.agent import loop
from academy.exchange import LocalExchangeFactory

class CountingCounter(Agent):
    count: int

    async def agent_on_startup(self) -> None:
        self.count = 0

    @loop
    async def increment(self, shutdown: asyncio.Event) -> None:
        while not shutdown.is_set():
            await asyncio.sleep(1)
            self.count += 1

    @action
    async def get_count(self) -> int:
        return self.count


async def main() -> int:

    async with await Manager.from_exchange_factory(
        factory=LocalExchangeFactory(),
        executors=ThreadPoolExecutor(),
    ) as manager:
        agent = await manager.launch(CountingCounter)

        print('Waiting 5s for agent loops to execute...')
        await asyncio.sleep(5)

        count = await agent.get_count()

        print(f"Agent loop executed {count} time(s)")

    return 0
    
# In a Jupyter notebook, we use top-level `await` to run the async main() coroutine
# because the notebook already has an event loop running.
await main()

###  4. Agent-Agent Communication: A Multi-Agent System

Academy allows you to build multi-agent systems. Agent Handles can be passed to other agents (or created by other agents) to allow one agent to invoke another.

In this example we implement three agents: a Coordinator agent, a Lowerer agent, and a Reverser agent.  The Coordinator manages the mutli-agent system, the Lowerer converts a string from uppercase to lowercase, and the Reverser reverses the characters in a string.  Each exposes an action that can be called by a client or another agent. 

In [ ]:
from academy.agent import action
from academy.agent import Agent
from academy.handle import Handle

class Lowerer(Agent):
    @action
    async def lower(self, text: str) -> str:
        return text.lower()

class Reverser(Agent):
    @action
    async def reverse(self, text: str) -> str:
        return text[::-1]
        
class Coordinator(Agent):
    def __init__(
        self,
        lowerer: Handle[Lowerer],
        reverser: Handle[Reverser],
    ) -> None:
        super().__init__()
        self.lowerer = lowerer
        self.reverser = reverser

    @action
    async def process(self, text: str) -> str:
        text = await self.lowerer.lower(text)
        text = await self.reverser.reverse(text)
        return text

We now use Academy to deploy the multi-agent system locally (later in the tutorial we show how to deploy this multi-agent system on remote resources). We deploy three agents (coordinator, lowerer, and reverser) passing handles from each to the coordinator. We then pass the coordinator some input text and allow the multi-agent system to process that input.  

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from academy.exchange import LocalExchangeFactory
from academy.manager import Manager

async def main() -> int:

    async with await Manager.from_exchange_factory(
        factory=LocalExchangeFactory(),
        executors=ThreadPoolExecutor(),
    ) as manager:
        lowerer = await manager.launch(Lowerer)
        reverser = await manager.launch(Reverser)
        coordinator = await manager.launch(
            Coordinator,
            args=(lowerer, reverser),
        )

        text = 'DRAWER'
        print(f'Invoking process("{text}") on {coordinator.agent_id}')
        result = await coordinator.process(text)
        print(f'Received result: "{result}"')
    
    return 0
    
# In a Jupyter notebook, we use top-level `await` to run the async main() coroutine
# because the notebook already has an event loop running.
await main()

### 5. LLM-powered Agents: Computing Materials Properties

###### Note: this example uses LanChain for interacting with an LLM and requires access to an OpenAI compatible LLM API. 

This agent-of-agents example illustrates an LLM-powered orchestration pattern in which an LLM-powered orchestrator delegates scientific work to a simulation agent via Academy actions. 

The LangChain agent framework is used to handle reasoning and tool selection; Academy supplies the agent lifecycle, communication, and execution substrate that allows simulation agents to be instantiated and accessed across heterogeneous environments.

Running this example requires langchain and langchain-openai.
````
pip install langchain>=1.0
pip install langchain-openai
````

#### Agents
The program uses two Academy agents:

Orchestrator: which embeds a LLM-powered LangChain agent that processes user requests and decides what tools to call via a ReAct-style loop. (The LangChain agent uses the OpenAI API to access the LLM.)

MySimAgent, a simulation agent that computes a molecular property

The Orchestrator interprets natural-language questions, decides when to invoke tools (by invoking MySimAgent), and aggregates results from simulation agents.

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.tools import tool

from academy.agent import action
from academy.agent import Agent
from academy.handle import Handle


class MySimAgent(Agent):
    """Agent for running tools to characterize molecules."""

    @action
    async def compute_ionization_energy(self, smiles: str) -> float:
        """Compute the ionization energy for the given molecule."""
        return 0.5


def make_sim_tool(handle: Handle[MySimAgent]):
    """Wraps an academy handle in a langchain tool. """

    @tool
    async def compute_ionization_energy(smiles: str) -> float:
        """Compute the ionization energy of a molecule."""
        return await handle.compute_ionization_energy(smiles)

    return compute_ionization_energy


# An Academy agent that creates a LangChain agent that will respond to
# questions about molecules by running a ReACT loop
class Orchestrator(Agent):
    """An Academy agent that creates a LangChain agent that will 
    respond to questions about molecules by running a ReACT loop"""
    
    def __init__(
        self,
        model: str,
        access_token: str,
        simulators: list[Handle[MySimAgent]],
        base_url: str | None = None,
    ):
        self.model = model
        self.access_token = access_token
        self.base_url = base_url
        self.simulators = simulators

    async def agent_on_startup(self) -> None:
        llm = ChatOpenAI(
            model=self.model,
            api_key=self.access_token,
            base_url=self.base_url,
        )

        tools = [make_sim_tool(agent) for agent in self.simulators]
        
        # The following call creates the LangChain agent
        self.react_loop = create_agent(llm, tools=tools)

    @action
    async def answer(self, goal: str) -> str:
        """Use other agents to answer questions about molecules."""
        
        return await self.react_loop.ainvoke(
            {'messages': [{'role': 'user', 'content': goal}]},
        )

#### Orchestration program

We now use Academy to deploy the agentic system locally. The Orchestrator uses a LangChain agent to interact with an OpenAI model and call the local simulation tool. The code below provides the connection details to the model and parses the output to show the response from the LLM and the tool call. 

In [ ]:
async def main() -> int:

    model = 'gpt-oss-120b' 
    token = ''
    url = 'https://inference-api.alcf.anl.gov/resource_server/metis/api/v1'
    
    async with await Manager.from_exchange_factory(
        factory=LocalExchangeFactory(),
    ) as manager:
        simulator = await manager.launch(MySimAgent)
        orchestrator = await manager.launch(
            Orchestrator,
            kwargs={
                'model': model,
                'access_token': token,
                'simulators': [simulator],
                'base_url': url,
            },
        )

        question = 'What is the simulated ionization energy of benzene?'

        data = await orchestrator.answer(question)

        # Print the results cleanly
        for m in data["messages"]:
            if hasattr(m, "tool_calls") and m.tool_calls:
                for tc in m.tool_calls:
                    print("Tool call:", tc["name"], tc["args"])
            elif hasattr(m, "tool_call_id"):
                print("Tool output:", m.name, m.content)
            elif m.type == "human":
                print("Query:", m.content)
            elif m.type == "ai" and m.content:
                print("Final:", m.content)
    return 0
    
# In a Jupyter notebook, we use top-level `await` to run the async main() coroutine
# because the notebook already has an event loop running.
await main()

### 6. Distributing Agents
Academy uses a cloud hosted exchange and Globus Compute to deploy and orchestrate distributed agentic systems. 

Here we use the GCExecutor (Globus Compute Executor) to deploy the agents remotely using Globus Compute. Globus Compute is a widely used FaaS platform that allows you to run Pyhton functions on arbitrary remote computing resources. 

We use the HttpExchange using our cloud-hosted exchange. This exchange relies on Globus Auth. When running the following code you will need to first authenticate to give permission for your Academy program to use the exchange.

Note: we use the same agents defined above in step 3.  

Note: you will need to have academy-py installed in your Globus Compute environment. E.g., to create a docker container with a running endpoint

````
pip install globus-compute-endpoint
pip install academy-py
globus-compute-endpoint configure --multi-user false
globus-compute-endpoint start default
````

In [ ]:
from __future__ import annotations

import asyncio
import logging
import multiprocessing
import os
from concurrent.futures import ProcessPoolExecutor

from academy.agent import action
from academy.agent import Agent
from academy.exchange.cloud.client import HttpExchangeFactory
from academy.handle import Handle
from academy.manager import Manager
from globus_compute_sdk import Executor as GCExecutor

EXCHANGE_ADDRESS = 'https://exchange.academy-agents.org'
GLOBUS_COMPUTE_ENDPOINT = 'e80fd8ba-98d2-4f07-9f96-51830cbd7145'

async def main() -> int:

    executor = GCExecutor(GLOBUS_COMPUTE_ENDPOINT)

    async with await Manager.from_exchange_factory(
        factory=HttpExchangeFactory(
            EXCHANGE_ADDRESS,
            auth_method='globus',
        ),
        executors=executor,
    ) as manager:
        lowerer = await manager.launch(Lowerer)
        reverser = await manager.launch(Reverser)
        coordinator = await manager.launch(
            Coordinator,
            args=(lowerer, reverser),
        )

        text = 'DEADBEEF'
        expected = 'feebdaed'

        print(f"Invoking process({text}) on{coordinator.agent_id}")

        result = await coordinator.process(text)
        
        print(f"Received result: {result}")

    return 0
    
# In a Jupyter notebook, we use top-level `await` to run the async main() coroutine
# because the notebook already has an event loop running.
await main()